In [1]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.callbacks import TensorBoard
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.preprocessing.image import ImageDataGenerator

MODEL_NAME = "MegaClassifier_a"
VERSION_BASE = "v5"
EPOCHS = 10
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

DATASET_CSV = os.path.abspath("./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv")
DATASET_PATH = os.path.dirname(DATASET_CSV)

dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset['file_name'] = dataset['file_name'].apply(lambda x: os.path.join(DATASET_PATH, x))
dataset['binary_label'] = dataset['binary_label'].astype(str)

train_df = dataset[dataset['subset'] == 'train']
validation_df = dataset[dataset['subset'] == 'validation']
test_df = dataset[dataset['subset'] == 'test']

class_weights = compute_class_weight(class_weight="balanced",
                                     classes=np.unique(train_df['binary_label']),
                                     y=train_df['binary_label'])
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

def focal_loss(alpha=0.25, gamma=1.0):
    def loss(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        pt = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        loss = -K.mean(alpha * K.pow(1. - pt, gamma) * K.log(pt))
        return loss

    return loss

def train_and_evaluate():
    print(f"\n🔹 Entrenando con Loss Weighting (pesos de clase) 🔹")

    train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
    datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

    train_images = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    validation_images = datagen.flow_from_dataframe(
        dataframe=validation_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    test_images = datagen.flow_from_dataframe(
        dataframe=test_df,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
    )

    mobilenet_v2 = tf.keras.applications.MobileNetV2(
        weights="imagenet",
        include_top=False,
        input_shape=IMAGE_SHAPE,
    )
    mobilenet_v2.trainable = False

    model = tf.keras.Sequential([
        mobilenet_v2,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ], name=f"{MODEL_NAME}_{VERSION_BASE}")

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
        loss=focal_loss(alpha=0.25, gamma=1.0),
        metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), AUC(name="auc")],
    )

    start_time = time.time()
    history = model.fit(
        train_images,
        epochs=EPOCHS,
        validation_data=validation_images,
        class_weight=class_weight_dict,
        callbacks=[
            TensorBoard(log_dir=f"./logs/{MODEL_NAME}/{VERSION_BASE}"),
        ]
    )
    training_time = time.time() - start_time

    results = model.evaluate(test_images)

    # ✅ Guardar métricas y resultados
    history_df = pd.DataFrame(history.history)
    os.makedirs(f"./logs/{MODEL_NAME}/{VERSION_BASE}", exist_ok=True)
    history_df.to_csv(f"./logs/{MODEL_NAME}/{VERSION_BASE}/history_{VERSION_BASE}.0.csv", index=False)

    metric_names = history.model.metrics_names
    evaluation_results = {("test_" + name): value for name, value in zip(metric_names, results)}
    evaluation_results["training_time"] = training_time

    results_df = pd.DataFrame([evaluation_results])
    results_df.to_csv(f"./logs/{MODEL_NAME}/{VERSION_BASE}/results_{VERSION_BASE}.0.csv", index=False)

    print(f"\n📉 Loss: {results[0]:.4f}")
    print(f"🎯 Accuracy: {results[1]:.4%}")
    print(f"✅ Precision: {results[2]:.4%}")
    print(f"🔄 Recall: {results[3]:.4%}")
    print(f"📊 AUC: {results[4]:.4f}")
    print(f"⏳ Tiempo de entrenamiento: {training_time:.2f} segundos")

    return results_df


results_df = train_and_evaluate()

print("\n✅ ¡Entrenamiento con Loss Weighting completado!")


🔹 Entrenando con Loss Weighting (pesos de clase) 🔹
Found 17054 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.


2025-02-24 09:56:35.851063: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-02-24 09:56:35.851090: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-02-24 09:56:35.851095: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-02-24 09:56:35.851127: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-24 09:56:35.851286: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10


2025-02-24 09:56:37.456938: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


533/533 [==============================] - ETA: 0s - loss: 0.0355 - accuracy: 0.8762 - precision: 0.9059 - recall: 0.9377 - auc: 0.9286

2025-02-24 09:57:22.506885: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


533/533 [==============================] - 57s 104ms/step - loss: 0.0355 - accuracy: 0.8762 - precision: 0.9059 - recall: 0.9377 - auc: 0.9286 - val_loss: 0.0262 - val_accuracy: 0.9162 - val_precision: 0.9548 - val_recall: 0.9168 - val_auc: 0.9732
Epoch 2/10
533/533 [==============================] - 55s 102ms/step - loss: 0.0260 - accuracy: 0.9136 - precision: 0.9399 - recall: 0.9492 - auc: 0.9639 - val_loss: 0.0245 - val_accuracy: 0.9288 - val_precision: 0.9341 - val_recall: 0.9602 - val_auc: 0.9762
Epoch 3/10
533/533 [==============================] - 56s 105ms/step - loss: 0.0238 - accuracy: 0.9225 - precision: 0.9470 - recall: 0.9534 - auc: 0.9700 - val_loss: 0.0219 - val_accuracy: 0.9340 - val_precision: 0.9597 - val_recall: 0.9397 - val_auc: 0.9802
Epoch 4/10
533/533 [==============================] - 55s 103ms/step - loss: 0.0224 - accuracy: 0.9278 - precision: 0.9519 - recall: 0.9551 - auc: 0.9735 - val_loss: 0.0206 - val_accuracy: 0.9382 - val_precision: 0.9534 - val_recall: 